## Intro

This is our final lecture!! Crazy I know. Today's class:
1. has the main goal of creating 3 visualizations and writing a story;
2. is built on what we have done during the last two weeks. Thus, it is really important that you finish up the work from previous weeks.

Until now we have mainly focused on exploratory data analysis and visualization. In the last couple of weeks we have also built a classification method and debiased it after having discovered that its predictions were not fair. This work was loosly reproducing previous work from [ProPublica](https://www.propublica.org/article/machine-bias-risk-assessments-in-criminal-sentencing). 

Today, we are going to put everything together to create a story about this case with our own analysis: 

1. First, we will get an introduction on explanatory visualizations and narratives;
2. Second, we will transform this notebook into an article, using both visualizations and descriptions to communicate your results to others.

## Part 1. Explanatory and narrative data visualization

We start with a video from Sune and then answer a few questions to make sure you've read the scientific article about types of explanatory dataviz. (*The video is from an old version of the class that used D3, so just ignore those parts. Sune will make a new one ASAP*).

[![IMAGE ALT TEXT HERE](https://img.youtube.com/vi/yHKYMGwefso/0.jpg)](https://www.youtube.com/watch?v=yHKYMGwefso)

> *Exercises 1.1*: Explanatory data visualization.
> * What are the three key elements to keep in mind when you design an explanatory visualization? List and explain them.
> * Explain in your own words: How is explanatory data analysis different from exploratory data analysis?

> *Exercise 1.2*: Answer a couple of questions from E. Segel and J. Heer paper [Narrative Visualization: Telling Stories with Data](http://vis.stanford.edu/files/2010-Narrative-InfoVis.pdf).
> 
> * What is the *Oxford English Dictionary's* defintion of a narrative?
> * What is your favorite visualization among the examples in section 3? Explain why in a few words.
> * Use Figure 7 to find the most common design choice within each category for the Visual narrative and Narrative structure (the categories within visual narrative are 'visual structuring', 'highlighting', etc).
> * Check out Figure 8 and section 4.3. What is your favorite genre of narrative visualization? Why? What is your least favorite genre? Why?

## Part 2. Communicate your story and results to others

#### Introduction

You have been hired by ProPublica as a freelencer to follow a case related to COMPAS (Correctional Offender Management Profiling for Alternative Sanctions), a popular commercial algorithm used by judges and parole officers for scoring criminal defendant’s likelihood of reoffending (recidivism).

You have collected the data and did some exploratory data analysis together with some modeling to understand what was going on. You have found that not only the data is biased, but also that this bias affects the performance of your classification algorithm. You know that this is a main issue and want to write an article that ProPublica will publish to inform the general public. In the article, you will explain the data bias, the algorithm prediction bias, and, because you are super thorough, you will also show how to debias the predictions of the algorithm.

---

#### Instruction to complete the article

These are the instructions to move forward:

1. First, to build a story I like to start by thinking about the key content: visualizations. What you are going to visualize will shape your story. I gave you instructions on how to build the visualizations below. We will make three visualizations: 
   * An interactive visualization to show the **bias in the data**
   * A visualization to show the **biased predictions** of your model
   * A visualization to show the **debiased results**
2. Second, we are going to convert the Notebook into an article. I drafted the structure of the article below, which you will replace with your content. These are the main parts: 
    * **A title** and **subtitles** (i.e. replace **Introduction**, **Results from exploratory data analysis**, etc. with a subtitle).
    * **An introduction**. 
    * **A description of** your findings from the **data exploration** step. 
    * **A description of** your findings from the **model predictions**.
    * **A description of** your results on **the debiasing step**.
    * **Limitations and Conclusion**.
    * **References.**

---

#### Instructions to create the visualizations

Here, you have the instructions to build your visualizations. Once you have them ready, move them to the article draft below!

**Note** Some of you where confused by the code in markdown, and Bokeh in general. Thus, I've created an [extra notebook](https://github.com/suneman/socialdata2022/blob/main/lectures/W7-8_ExtraBokeh.ipynb) with the structure of the code that create the step 1 step 2 and compute the fractions of step 3 of *Visualization 1* below. You need to fill in the code, but hopefully it will be clearer than its markdown version from Week 7. Also, you can reuse the code to complete Week 7 Ex. 1.1.

> *Visualization 1.* Exploratory data analysis. The interactive visualization we are going to make uses the same code you put together for Week 7 Exercise 1.1 (where we made bar plot of the number of recommited crimes across ages divided by race), with a few changes/adds on:
>
>    * First, instead of age, create an `age_group` column to use on the x-axis. The groups should be $(0,20],(20,40],(40,60],(60,80],(80,100]$. 
>    * Second, instead of using the number of recommitted crimes for each race, compute the total number of samples per age group and the total number of recommitted crimes per age group. 
>    * Third, we are going to add a `HoverTool` as in Week 7, Exercise 1.2 showing two fractions:
>        * the number of recommitted crimes of the age group over the number of samples of the age group;
>        * the number of recommitted crimes of the age group over the total recommitted crimes across age groups.
>    * The result should look similar to [this](https://raw.githubusercontent.com/suneman/socialdata2022/main/files/W8_Viz1_sub.png).
>    * Now that you have the code for this plot, create other two plots in the same way but by using `sex` and `race`.
>    * Finally, combine the three plots into one unique visualization with `Tabs` and `Panel`. To have an hint on how to do it, have a look at [this solution](https://stackoverflow.com/questions/38169164/adding-layout-to-tabs-on-bokeh-dashboard)
>    * You just created your first visualization for the article! You can personalize it as you like, but it should look somewhat like the one in the following video: 

[![IMAGE ALT TEXT HERE](https://img.youtube.com/vi/eJZGQiA7CKo/0.jpg)](https://www.youtube.com/watch?v=eJZGQiA7CKo)

In [1]:
from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.layouts import row
from bokeh.plotting import figure
from bokeh.palettes import all_palettes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
output_notebook()

Loading BokehJS ...

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/suneman/socialdata2022/main/files/recidivism_dataset_sub.csv')
preds = ['age', 'sex', 'race', 'juv_fel_count', 'juv_misd_count', 'juv_other_count', 'priors_count', 'is_recid', 'days_b_screening_arrest', 'c_charge_degree']
resp = df['two_year_recid']
df = df[preds + ['two_year_recid']]

df = df[df['is_recid'] != -1]
df = df[df['c_charge_degree'] != 'O']
df = df[(df['days_b_screening_arrest'] > -30) & (df['days_b_screening_arrest'] < 30)]
df = df.drop(['is_recid', 'c_charge_degree', 'days_b_screening_arrest'], axis=1)



In [70]:
bins = [0,21,41,61,81,101]
labels = ['<=20', '21-40', '41-60', '61-80', '81-100']
df['age_group'] = pd.cut(df['age'], bins=bins, labels=labels, right=False)

samples = df['age_group'].value_counts().sort_index()
recommits = df.groupby('age_group')['two_year_recid'].sum().sort_index()
frac = recommits / samples
tot_frac = recommits / recommits.sum()

d = {'age_group': list(range(len(labels))), 'n_samples': list(samples.values), 'n_recommits': list(recommits.values), 'fraction': list(frac.values), 'tot_frac': list(tot_frac)}

source = ColumnDataSource(data=d)


p1 = figure(width=600, height=400, title='Crimes per age group')

bar = {} # to store vbars 
#colors = all_palettes['Pastel1'][3]
colors = ['#fbb4ae', '#b3cde3']

for indx, i in enumerate(['n_samples', 'n_recommits']):
    bar[i] = p1.vbar(x='age_group', width=0.5, top=i, source=source, legend_label=i, color=colors[indx])

p1.legend.click_policy = 'hide' #assigns the click policy (you can try to use 'hide')

TOOLTIPS = [
    ("Fraction of recommits for age group", "@fraction"),
    ("Fraction of recommits across all age groups", '@tot_frac')
]

p1.add_tools(HoverTool(tooltips=TOOLTIPS))

In [71]:
samples = df['sex'].value_counts().sort_index()
recommits = df.groupby('sex')['two_year_recid'].sum().sort_index()
frac = recommits / samples
tot_frac = recommits / recommits.sum()

d = {'sex': list(range(len(set(df['sex'])))), 'n_samples': list(samples.values), 'n_recommits': list(recommits.values), 'fraction': list(frac.values), 'tot_frac': list(tot_frac)}

source = ColumnDataSource(data=d)


p2 = figure(width=600, height=400, title='Crimes per sex')

bar = {} # to store vbars 
#colors = all_palettes['Pastel1'][3]
colors = ['#fbb4ae', '#b3cde3']

for indx, i in enumerate(['n_samples', 'n_recommits']):
    bar[i] = p2.vbar(x='sex', width=0.5, top=i, source=source, legend_label=i, color=colors[indx])

p2.legend.click_policy = 'hide' #assigns the click policy (you can try to use 'hide')

TOOLTIPS = [
    ("Fraction of recommits for sex", "@fraction"),
    ("Fraction of recommits across all sexes", '@tot_frac')
]

p2.add_tools(HoverTool(tooltips=TOOLTIPS))

In [72]:
samples = df['race'].value_counts().sort_index()
recommits = df.groupby('race')['two_year_recid'].sum().sort_index()
frac = recommits / samples
tot_frac = recommits / recommits.sum()

d = {'race': list(range(len(set(df['race'])))), 'n_samples': list(samples.values), 'n_recommits': list(recommits.values), 'fraction': list(frac.values), 'tot_frac': list(tot_frac)}

source = ColumnDataSource(data=d)


p3 = figure(width=600, height=400, title='Crimes per race')

bar = {} # to store vbars 
#colors = all_palettes['Pastel1'][3]
colors = ['#fbb4ae', '#b3cde3']

for indx, i in enumerate(['n_samples', 'n_recommits']):
    bar[i] = p3.vbar(x='race', width=0.5, top=i, source=source, legend_label=i, color=colors[indx])

p.legend.click_policy = 'hide' #assigns the click policy (you can try to use 'hide')

TOOLTIPS = [
    ("Fraction of recommits for race", "@fraction"),
    ("Fraction of recommits across all races", '@tot_frac')
]

p3.add_tools(HoverTool(tooltips=TOOLTIPS))


p3.xaxis.ticker = d['race']
races = list(set(df['race'])) 
p3.xaxis.major_label_overrides = {i: frac.index[i] for i in range(max(d['race']) + 1)}

In [77]:
from bokeh.layouts import layout
from bokeh.models.widgets import Tabs, Panel
from bokeh.io import curdoc

l1 = layout([[p1]], sizing_mode='fixed')
l2 = layout([[p2]], sizing_mode='fixed')
l3 = layout([[p3]],sizing_mode='fixed')

tab1 = Panel(child=l1,title="This is Tab 1")
tab2 = Panel(child=l2,title="This is Tab 2")
tab3 = Panel(child=l3,title="This is Tab 3")
tabs = Tabs(tabs=[tab1, tab2, tab3])

curdoc().add_root(tabs)
show(tabs)

> *Visualization 2.* Classification results. This visualization will show the results of your classification model and how they are biased. This is just a different way of visualizing the results from Week 7, Exercise 2. Please, use the model you have trained/tested in Week 6 by using only the columns selected in Week 6, Exercise 3.1 and by preprocessing as in Exercise 3.2. We will make a plot containing three subplots:
> 
>   1. The first subplot shows the feature importances from the model.
>      * **If you used a Decision Tree**, use [`feature_importances_`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) to get the values and plot them as a [horizontal lollipop plot](https://python-graph-gallery.com/182-vertical-lollipop-plot).
>      * **If you used a Random Forest** you can also have the standard deviation for each feature importance by following [these steps](https://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_importances.html). Plot the feature importances and standard deviations by using [`plt.errorbar()`](https://matplotlib.org/3.5.1/api/_as_gen/matplotlib.pyplot.errorbar.html).
>   2. The second subplot shows the confusion matrix for the entire model (as in Week 7 Ex. 2). 
>      * **N.B.** Please, use the `confusion_matrix()` function from `sklearn.metrics` and **make sure you add the parameter** `normalize="true"` which normalizes the rows of the confusion matrix and will help the comparison in the third subplot.
>      * **Hint.** [Here](https://stackoverflow.com/questions/19233771/sklearn-plot-confusion-matrix-with-labels) and hint on how to plot it.
>   3. The third subplot shows the difference between the values of the confusion matrices of the two populations we are interested in, i.e. African-Americans and Caucasian, and the confusion matrix of your model:
> $$D_{aa} = CM_{aa} - CM_{tot};$$
> $$D_{ca} = CM_{ca} - CM_{tot};$$
> where $CM_{aa}$, $CM_{ca}$, and $CM_{tot}$ are the confusion matrices for African-Americans, Caucasians, and the total one from the model respectively. **N.B.** You do not have to retrain the model to get $CM_{aa}$ and $CM_{ca}$.
>      * Display the values for the difference in TP, TN, FP, and FN in an [horizontal bar plot](https://matplotlib.org/stable/gallery/lines_bars_and_markers/barh.html), where you have the difference on the x-axis, and the TP, TN, FP and FN labels on the y-axis. You can transform the difference in percentage if you wish.
>   4. Finally, customize the plot as you like: choose colors, add labels, titles, legends, captions etc. [Here](https://raw.githubusercontent.com/suneman/socialdata2022/main/files/W8_Viz2_custom.png) and [here](https://raw.githubusercontent.com/suneman/socialdata2022/main/files/W8_Viz2_lollipop.png) you can have a look at how I made them, but it is just for inspiration do not make it exactly the same...make it yours! make it better! 😊

> *Visualization 3.* Debiasing results. The plot you are going to make shows the results from the debiasing method. Here, you will use the *Equal Odds* method and the results from Week 7, Ex. 3.3. The plot consists of two subplots:
>   1. The first subplot shows the AUC-ROC curve and highlights the threshold found for the model. It is the same visualization of Week 7, Ex. 3.3 but by using matplotlib, seaborn, etc. instead of Bokeh. In my version of the plot below, two things are missing (I want you to chose the points on the curve 😇) so **make sure to add them**:
>      * Mark the points you chose on the curves, you can for instance highlight them with a different color or symbol, or you can use a [circle](https://stackoverflow.com/questions/9215658/plot-a-circle-with-pyplot) or point at them with [arrows](https://stackoverflow.com/questions/53538909/arrow-in-plot-matplotlib-pyplot)... it's up to you.
>      * Add a text on the figure that display the thresholds chosen. You can add texts on matplotlib figures `plt.figtext()`.
>   2. The second subplot shows the True-Positive and False-Positive rates for African-Americans and Caucasian before and after the debiasing method. You can do this with a bar plot of course, however I was a bit bored of bar plots and decided that we will use a [slope chart](https://towardsdatascience.com/slope-charts-why-how-11c2a0bc28be#:~:text=Slope%20charts%20are%20simple%20graphs,in%20the%20analyzed%20time%20interval.) instead. 
>      * The slope chart will have 4 lines: TPR for African-Americans, TPR for Caucasian, FPR for African-Americans, and FPR for Caucasian. Each line has two points, i.e. the value of the rate before and after debiasing. [Here](https://raw.githubusercontent.com/suneman/socialdata2022/main/files/W8_Viz3_custom.png), you can find my version of the plot. Again, customize it as you like, make it yours! make it better! 😊
>      * **Hint** You can have a look at this [website](https://www.machinelearningplus.com/plots/top-50-matplotlib-visualizations-the-master-plots-python/#18.-Slope-Chart) for inspiration, it also include many other visualization types so check it out for your project! 

**Note** You can add anything you like (highlighting areas, adding text etc.) to the visualizations to make them more explanatory! 

Now that you have the visualizations, move them in the structure of the article below and write the article. You should include your code, but if you want to hide the input cells to make your notebook look like an article you can do it by installing an [nbextension](https://jupyter-contrib-nbextensions.readthedocs.io/en/latest/install.html)!

# ADD YOUR TITLE HERE

**Introduction.** Write your introduction here. Think about the audience. This is an article for the general public, so you should try to make it engaging, relatable, not too technical etc.

In [ ]:
#First viz here

**Results from exploratory data analysis.** Write this section here. The aim is to show that the data is biased, which might have an impact on the model. Relate this part to the first visualization.

In [ ]:
#Second viz here

**Results from the classification task.** Write this section here. The aim is to show that the model is biased as it is learning from biased data. Relate this part to the second visualization.

In [ ]:
#third viz here

**Results from debiasing method.** Write this section here. The aim is to show that there are methods that can help to enhance fairness in cases as this one. Relate this part to the third visualization.

**Limitations and conclusions.** Write this part here. These are your main take aways, the end of your story, where you should also highlight possible implications/limitations of what you presented.

**References.** Any article need good sources! Whenever you make a statement, refer to data sources, previous studies, methods, etc. add a reference. 

1. reference 1
2. reference 2
3. reference 3
4. etc.

You made it! This was the last lecture for this class! 

Before you go, please:
1. <mark> take a minute of your time (it is really one minute) to fill this [form](https://forms.gle/4efUQXZbjHdCaSsP8). </mark>
2. <mark> help us understand what were the pros/cons of this class by filling the second [Questionnaire](https://forms.gle/25YyJZJuBJgzXGUe8) of the course!

Thank you so much! 